In [1]:
import matplotlib
matplotlib.use('TkAgg') # Set Matplotlib backend to TkAgg (for interactive plotting on some systems)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation # to animate particle dynamics

from shapely.geometry import LineString, Point # code for computing link geometry (i.e. edges in network)
from shapely.strtree import STRtree 
from collections import defaultdict 
import networkx as nx # graph software
import random

plt.rc("text", usetex=False)
plt.rc("font", family = "serif")
plt.rc("figure", figsize=(10,8))
%config InlineBackend.figure_format = 'retina'

In [2]:
# define a class for particle objects
class Particle:
    def __init__(self, x, y, p_run, p_tumble, theta, speed, p_link, L_link_mean, L_link_std):
        self.position = np.array([x, y])  # Current position (in unit square)
        self.p_run = p_run  # Probability of running at each timestep
        self.p_tumble = p_tumble  # Probability of tumbling (direction change)
        self.theta = theta  # Current heading (angle in radians)
        self.speed = speed  # Speed per timestep
        self.p_link = p_link  # Probability of laying a link at each timestep
        self.L_link_mean = L_link_mean  # Mean link length
        self.L_link_std = L_link_std  # Std dev of link length

    def maybe_lay_link(self):
        if np.random.rand() < self.p_link:
            # Sample link length from Gaussian, fold negative to positive
            L_link = np.abs(np.random.normal(self.L_link_mean, self.L_link_std))
            return True, L_link
        return False, 0.0

    def run(self, dt):
        if np.random.rand() < self.p_run:
            displacement = self.speed * dt # how far does the particle run?
            dx = displacement * np.cos(self.theta) # change in x coordinate (w/ orientation)
            dy = displacement * np.sin(self.theta) # change in y coordinate (w/ orientation)
            self.position = (self.position + np.array([dx, dy])) % 1.0  # Enforce periodic boundaries

    def tumble(self):
        if np.random.rand() < self.p_tumble:
            self.theta = np.random.uniform(-np.pi, np.pi)  # Randomly sample new direction

# laying down material + history of particle locations
edges = []  # List of edges as ((x1, y1), (x2, y2)) tuples
nodes = []  # List of node positions (redundant in current logic)

def add_link(particle, L_link):
    # Compute the endpoints of the link centered at particle's position and aligned with heading
    dx = 0.5 * L_link * np.cos(particle.theta)
    dy = 0.5 * L_link * np.sin(particle.theta)
    start = (particle.position - np.array([dx, dy])) % 1.0  # Periodic wrap
    end = (particle.position + np.array([dx, dy])) % 1.0 # Periodic wrap
    edges.append((start, end))  # Store edge
    nodes.append(particle.position.copy())  # Currently adds a node at every deposition (may not be desired)

def simulate(N, steps, dt, particle_params):
    global nodes, edges
    nodes, edges = [], []  # Clear previous run data
    
    # Create N particles with randomized initial positions and angles
    particles = [
        Particle(
            x=np.random.rand(), y=np.random.rand(),  # Random position
            theta=np.random.uniform(-np.pi, np.pi),  # Random direction
            **particle_params  # Fill in other parameters
        )
        for _ in range(N)
    ]

    for step in range(steps):
        for p in particles:
            laid_link, L_link = p.maybe_lay_link()
            if laid_link:
                add_link(p, L_link)  # Add link if randomly chosen
            p.run(dt)  # Move forward (maybe)
            p.tumble()  # Change direction (maybe)

    # Placeholder for optional post-processing (like merging links)
    #edges[:] = merge_collinear_links(edges)

def plot_network(nodes, edges, step=None):
    plt.figure(figsize=(6, 6))
    for start, end in edges:
        if np.linalg.norm(np.array(start) - np.array(end)) > 0.5:
            continue  # Skip links that wrap around the periodic boundary
        x_vals = [start[0], end[0]]
        y_vals = [start[1], end[1]]
        plt.plot(x_vals, y_vals, 'b-', alpha=0.5)  # Draw edge as blue line

    if nodes:
        xs, ys = zip(*nodes)
        plt.scatter(xs, ys, color='red', s=10, label='Nodes')  # Draw nodes as red dots

    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.gca().set_aspect('equal')  # Keep square aspect ratio
    plt.title(f"Network at Step {step}" if step is not None else "Network")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.show()

In [3]:
def build_binned_network(edges, L_link_mean, tol=1e-5):
    box_size = L_link_mean
    M = int(np.ceil(1.0 / box_size))

    def get_box_indices(line):
        minx, miny, maxx, maxy = line.bounds
        ix_min, iy_min = int(minx // box_size), int(miny // box_size)
        ix_max, iy_max = int(maxx // box_size), int(maxy // box_size)
        return [(ix, iy)
                for ix in range(ix_min, ix_max + 1)
                for iy in range(iy_min, iy_max + 1)]

    def round_point(pt):
        return (round(pt.x, 5), round(pt.y, 5))

    # Step 1: Convert edge list into index-labeled LineStrings
    lines = []
    line_index = {}  # index → LineString
    grid = defaultdict(list)  # (ix, iy) → list of line indices

    for i, (start, end) in enumerate(edges):
        line = LineString([tuple(start), tuple(end)])
        lines.append(line)
        line_index[i] = line
        for box_idx in get_box_indices(line):
            grid[box_idx].append(i)
            
    # Step 2: Detect intersections per line with containment check
    intersections = defaultdict(set)  # line index → set of intersection points

    for ix in range(M):
        for iy in range(M):
            local_ids = []
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    neighbor = (ix + dx, iy + dy)
                    if neighbor in grid:
                        local_ids.extend(grid[neighbor])

            local_ids = list(set(local_ids))
            for i in range(len(local_ids)):
                for j in range(i + 1, len(local_ids)):
                    id1, id2 = local_ids[i], local_ids[j]
                    line1, line2 = line_index[id1], line_index[id2]
                    inter = line1.intersection(line2)
                    if isinstance(inter, Point):
                        pt = round_point(inter)
                        # Only add pt if it truly lies on the line segment
                        if line1.buffer(tol).contains(inter):
                            intersections[id1].add(pt)
                        if line2.buffer(tol).contains(inter):
                            intersections[id2].add(pt)

    # Step 3: Build graph from valid per-line intersections
    G = nx.Graph()

    for pts in intersections.values():
        for pt in pts:
            G.add_node(pt)

    for idx, pts in intersections.items():
        line = line_index[idx]
        pts_filtered = []
        for pt in pts:
            proj = line.project(Point(pt))
            if 0 <= proj <= line.length:
                pts_filtered.append(pt)

        if len(pts_filtered) < 2:
            continue

        pts_filtered.sort(key=lambda pt: line.project(Point(pt)))
        for i in range(len(pts_filtered) - 1):
            p1, p2 = pts_filtered[i], pts_filtered[i + 1]
            dist = np.linalg.norm(np.array(p1) - np.array(p2))
            G.add_edge(p1, p2, weight=dist)

    return G

def plot_graph(G):
    pos = {node: node for node in G.nodes()}  # Position = coordinate
    nx.draw(G, pos, node_size=30, node_color='red', edge_color='blue', with_labels=False)
    plt.gca().set_aspect('equal')
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.show()

In [4]:
# 1. Define particle behavior parameters
params = {
    'p_run': 0.5,           # 50% chance of running
    'p_tumble': 0.5,        # 50% chance of tumbling
    'speed': 0.05,          # Distance per time step
    'p_link': 0.2,          # 20% chance of laying a link per step
    'L_link_mean': 0.1,     # Average link length
    'L_link_std': 0.02      # Std deviation of link length
}

# 2. Run the simulation
steps = 100
N = 10
dt = 1.0

simulate(N=N, steps=steps, dt=dt, particle_params=params)

# 3. Optional: Visualize the raw links and deposition points
plot_network(nodes, edges, step=steps)

# 4. Build the network via binned spatial intersection detection
G = build_binned_network(edges, L_link_mean=params['L_link_mean'])

# 5. Visualize the final network graph
plot_graph(G)

/opt/anaconda3/lib/python3.13/site-packages/shapely/set_operations.py:168: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/opt/anaconda3/lib/python3.13/site-packages/shapely/set_operations.py:168: RuntimeWarning: divide by zero encountered in intersection
  return lib.intersection(a, b, **kwargs)


In [5]:
def compute_alpha(q, dt):
    return -np.log(1 - q) / dt

def compute_q_from_alpha(alpha, dt):
    return 1 - np.exp(-alpha * dt)

def compute_v_from_D_alpha(D, alpha):
    return np.sqrt(2 * alpha * D)

# === Network Stats ===
def compute_graph_stats(G):
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    degrees = [deg for _, deg in G.degree()]
    avg_deg = np.mean(degrees) if degrees else 0
    return {
        'nodes': num_nodes,
        'edges': num_edges,
        'avg_degree': avg_deg,
    }

In [6]:
def run_fixed_alpha_sweep(alpha_fixed, v_values, base_params, N, steps, dt):
    results = []
    print(f"\n=== Regime 1: Fixed alpha = {alpha_fixed} ===")
    for v in v_values:
        params = base_params.copy()
        params['speed'] = v
        params['p_tumble'] = compute_q_from_alpha(alpha_fixed, dt)

        simulate(N=N, steps=steps, dt=dt, particle_params=params)
        G = build_binned_network(edges, L_link_mean=params['L_link_mean'])

        stats = compute_graph_stats(G)
        stats.update({'v': v, 'alpha': alpha_fixed, 'regime': 'fixed_alpha'})
        results.append(stats)

        plot_graph(G)
        print(f"v = {v:.3f} → Nodes: {stats['nodes']}, Edges: {stats['edges']}, ⟨k⟩ = {stats['avg_degree']:.2f}")

    return results

In [7]:
def run_fixed_D_sweep(D_fixed, alpha_values, base_params, N, steps, dt):
    results = []
    print(f"\n=== Regime 2: Fixed D = {D_fixed} ===")
    for alpha in alpha_values:
        params = base_params.copy()
        params['p_tumble'] = compute_q_from_alpha(alpha, dt)
        params['speed'] = compute_v_from_D_alpha(D_fixed, alpha)

        simulate(N=N, steps=steps, dt=dt, particle_params=params)
        G = build_binned_network(edges, L_link_mean=params['L_link_mean'])

        stats = compute_graph_stats(G)
        stats.update({'alpha': alpha, 'D': D_fixed, 'regime': 'fixed_D'})
        results.append(stats)

        plot_graph(G)
        print(f"α = {alpha:.2f} → Nodes: {stats['nodes']}, Edges: {stats['edges']}, ⟨k⟩ = {stats['avg_degree']:.2f}")

    return results

In [8]:
base_params = {
    'p_run': 0.5,              # Always 0.5 for now
    'p_tumble': 0.5,           # Will be overwritten by q
    'speed': 0.05,             # Will be overwritten
    'p_link': 0.2,
    'L_link_mean': 0.1,
    'L_link_std': 0.02
}

# Simulation parameters
N = 20
steps = 100
dt = 1.0

# Sweep parameters
v_values = np.linspace(0.01, 0.1, 5)
alpha_fixed = 1.0  # Fixed for Regime 1

alpha_values = [0.5, 1.0, 2.0, 5.0]
D_fixed = 0.001  # Fixed for Regime 2

# Run sweeps
results_alpha = run_fixed_alpha_sweep(alpha_fixed, v_values, base_params, N, steps, dt)
results_D = run_fixed_D_sweep(D_fixed, alpha_values, base_params, N, steps, dt)


=== Regime 1: Fixed alpha = 1.0 ===
v = 0.010 → Nodes: 5927, Edges: 11466, ⟨k⟩ = 3.87


KeyboardInterrupt: 

In [ ]:
def plot_stat_vs_param(results, param_name, stat_name, title):
    values = [r[param_name] for r in results]
    stats = [r[stat_name] for r in results]
    plt.plot(values, stats, 'o-', label=stat_name)
    plt.xlabel(param_name)
    plt.ylabel(stat_name)
    plt.title(title)
    plt.grid(True)
    plt.show()

plot_stat_vs_param(results_alpha, 'v', 'nodes', 'Number of Nodes vs Speed (Fixed α)')
plot_stat_vs_param(results_D, 'alpha', 'nodes', 'Number of Nodes vs α (Fixed D)')